<h1>Treniranje modela linearne regresije</h1>

U ovom poglavlju prikazati će se treniranje modela linearne regresije, postupci procesiranja podataka i primjeri rezultata.

<h2>Učitavanje potrebnih podataka</h2>

Potrebni podatci izvađeni su SQL upitom i spremljeni u datoteku. Spajanje je po ključu vremena cilja predikcije (stupac qty_future) i vremena cilja vremenske prognoze.

In [1]:

import pandas as pd

# Učitavanje spojenih podataka s najažurnijim prognozama
dataframe_all = pd.read_csv('Data/SEDrava1_Merged_All.csv')
dataframe_all


,tof,vt,production_timestamp,barometer,outtemp,windspeed,winddir,rain,radiation,cloud_cover,...,qty_lag2,qty_lag3,qty_lag4,qty_lag5,qty_lag6,qty_lag7,qty_lag8,qty_day,qty_week,qty_future
0,2024-03-01 18:00:00,2024-03-01 23:00:00,2024-03-01 23:30:00,1008.4,9.9,2.3,129,0.0,0,82.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-03-01 18:00:00,2024-03-01 23:00:00,2024-03-01 23:15:00,1008.4,9.9,2.3,129,0.0,0,82.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-03-01 18:00:00,2024-03-01 23:00:00,2024-03-01 23:00:00,1008.4,9.9,2.3,129,0.0,0,82.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024-03-01 18:00:00,2024-03-01 23:00:00,2024-03-01 22:45:00,1008.4,9.9,2.3,129,0.0,0,82.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024-03-01 18:00:00,2024-03-01 22:00:00,2024-03-01 22:30:00,1008.6,10.8,3.9,122,0.0,0,90.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20155,2023-07-05 00:00:00,2023-07-05 01:00:00,2023-07-05 01:00:00,1015.6,17.4,1.0,336,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20156,2023-07-05 00:00:00,2023-07-05 01:00:00,2023-07-05 00:45:00,1015.6,17.4,1.0,336,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20157,2023-07-04 18:00:00,2023-07-05 00:00:00,2023-07-05 00:30:00,1015.9,18.2,2.4,2,0.0,0,60.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20158,2023-07-04 18:00:00,2023-07-05 00:00:00,2023-07-05 00:15:00,1015.9,18.2,2.4,2,0.0,0,60.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



Ovdje se ispisuje matrica korelacije. Vidi se velika korelacija ciljnog stupca s određenim stupcima, kao sunčevo zračenje, proizvodnja neposredno prije, i proizvodnja dan prije. Vidi se i očekivan odnos korelacij ciljnog stupca s njegovim neposrednim prethodnicima koji opada, te opet raste npr. dan prije kako je to bilo prošli dan u istom vremenu.


In [2]:
# Ispis matrice korelacije, za procjenu korelacije značajki
dataframe_all.corr( numeric_only=True)

,barometer,outtemp,windspeed,winddir,rain,radiation,cloud_cover,qty,qty_lag1,qty_lag2,qty_lag3,qty_lag4,qty_lag5,qty_lag6,qty_lag7,qty_lag8,qty_day,qty_week,qty_future
barometer,1.000000,-0.259283,-0.319434,0.043518,-0.129517,0.009270,-0.369699,0.061759,0.056463,0.051064,0.045687,0.040284,0.035093,0.029975,0.025135,0.020508,0.009799,-0.034873,0.067023
outtemp,-0.259283,1.000000,-0.034048,-0.145138,-0.008463,0.593214,-0.267305,0.504440,0.518151,0.530500,0.541244,0.550328,0.557602,0.563268,0.567244,0.569644,0.512547,0.492612,0.488983
windspeed,-0.319434,-0.034048,1.000000,0.150320,0.103318,0.068363,0.248243,0.062930,0.073009,0.081774,0.089165,0.095306,0.099084,0.101801,0.102282,0.101051,0.075742,0.125203,0.052109
winddir,0.043518,-0.145138,0.150320,1.000000,-0.002791,-0.057945,0.046845,-0.043413,-0.046838,-0.049676,-0.053123,-0.057647,-0.062502,-0.066870,-0.070554,-0.073831,-0.040928,-0.031876,-0.041408
rain,-0.129517,-0.008463,0.103318,-0.002791,1.000000,-0.065123,0.216481,-0.068003,-0.067529,-0.066426,-0.064850,-0.063478,-0.061811,-0.060389,-0.058427,-0.055826,-0.001578,-0.037625,-0.067747
radiation,0.009270,0.593214,0.068363,-0.057945,-0.065123,1.000000,-0.270787,0.902527,0.906406,0.904650,0.897343,0.884020,0.865197,0.842389,0.815206,0.783759,0.849815,0.814312,0.892283
cloud_cover,-0.369699,-0.267305,0.248243,0.046845,0.216481,-0.270787,1.000000,-0.278183,-0.278086,-0.277838,-0.277365,-0.276366,-0.274568,-0.271739,-0.268297,-0.264204,-0.168464,-0.131018,-0.277995
qty,0.061759,0.504440,0.062930,-0.043413,-0.068003,0.902527,-0.278183,1.000000,0.975391,0.951741,0.926300,0.897418,0.864545,0.828855,0.788687,0.745056,0.833544,0.786353,0.975391
qty_lag1,0.056463,0.518151,0.073009,-0.046838,-0.067529,0.906406,-0.278086,0.975391,1.000000,0.975391,0.951741,0.926300,0.897418,0.864545,0.828855,0.788687,0.832821,0.783581,0.951741
qty_lag2,0.051064,0.530500,0.081774,-0.049676,-0.066426,0.904650,-0.277838,0.951741,0.975391,1.000000,0.975391,0.951741,0.926300,0.897418,0.864545,0.828855,0.826029,0.774439,0.926300


Ovdje se radi razdvajanje na skup za učenje i testiranje. Kako bi se simuliralo to da će prilikom rada modela biti dostupni podatci iz prošlosti a nedostupni podatci iz budućnosti, zadnji mjesec će se odvojiti za testiranje, a ostalo će biti odvojeno za treniranje. Razdvajanje je napravljeno SQL upitom.

In [3]:

# Učitavanje datoteka s razdvojenim podatcima za predviđanje i testiranje i razdvajanje stupaca

dataframe_train = pd.read_csv('Data/SEDrava1_Merged_Training.csv')
dataframe_test = pd.read_csv('Data/SEDrava1_Merged_Test.csv')

X_train = dataframe_train[['barometer', 'outtemp', 'windspeed', 'winddir', 'rain', 'radiation', 'cloud_cover', 'qty', 'qty_lag1', 'qty_lag2', 'qty_lag3', 'qty_lag4', 'qty_lag5', 'qty_lag6', 'qty_lag7', 'qty_lag8', 'qty_day', 'qty_week']]
y_train = dataframe_train['qty_future']

X_test = dataframe_test[['barometer', 'outtemp', 'windspeed', 'winddir', 'rain', 'radiation', 'cloud_cover', 'qty', 'qty_lag1', 'qty_lag2', 'qty_lag3', 'qty_lag4', 'qty_lag5', 'qty_lag6', 'qty_lag7', 'qty_lag8', 'qty_day', 'qty_week']]
y_test = dataframe_test['qty_future']



Prvo se implementira model obične linearne regresije. Prije učenja modela podatci se skaliraju da se smanji utjecaj različitih skala veličine podataka.


In [4]:
# Definiranje modela i učenje
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

scaler_1 = StandardScaler()
scaler_1.fit(X_train)
X_train_scaled = scaler_1.transform(X_train)
reg_1 = LinearRegression().fit(X_train_scaled, y_train)


In [5]:
# Predviđanje, evaluiranje modela MAPE metrikom
from sklearn.metrics import mean_squared_error
from math import sqrt

X_test_scaled = scaler_1.transform(X_test)
y_pred = reg_1.predict(X_test_scaled)
sqrt(mean_squared_error(y_test, y_pred))

0.11131196917444171

In [6]:
#pd.Series(y_pred)

In [7]:
#y_test

Zatim se implementira model polinomijalne regresije stupnja 2. To se radi pretprocesiranjem podataka i zatim skaliranjem.

In [8]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2)
X_train_poly2 = poly.fit_transform(X_train)
X_test_poly2 = poly.fit_transform(X_test)

scaler_2 = StandardScaler()
scaler_2.fit(X_train_poly2)
X_train_poly2_scaled = scaler_2.transform(X_train_poly2)
#X_train_poly2_scaled

In [9]:
# Definiranje modela i učenje

reg_2 = LinearRegression().fit(X_train_poly2_scaled, y_train)


In [10]:
# Predviđanje, evaluiranje modela MAPE metrikom

X_test_poly2_scaled = scaler_2.transform(X_test_poly2)
y_pred = reg_2.predict(X_test_poly2_scaled)
sqrt(mean_squared_error(y_test, y_pred))

0.10993632896596889

In [11]:
#pd.Series(y_pred)

In [12]:
#y_test

Konačno, implementira se model polinomijalne regresije stupnja 3. 

In [13]:
poly = PolynomialFeatures(3)
X_train_poly3 = poly.fit_transform(X_train)
X_test_poly3 = poly.fit_transform(X_test)

scaler_3 = StandardScaler()
scaler_3.fit(X_train_poly3)
X_train_poly3_scaled = scaler_3.transform(X_train_poly3)
#X_train_poly2_scaled

In [14]:
# Definiranje modela i učenje

reg_3 = LinearRegression().fit(X_train_poly3_scaled, y_train)

In [15]:
# Predviđanje, evaluiranje modela MAPE metrikom

X_test_poly3_scaled = scaler_3.transform(X_test_poly3)
y_pred = reg_3.predict(X_test_poly3_scaled)
sqrt(mean_squared_error(y_test, y_pred))

0.13737021072366284

Vidi se da su svi rezultati treniranja relativno slični. Prosječna greška predviđanja za slijedeći korak je oko 0.1 MW, što je prihvatljiva preciznost. Vidimo da uvođenje novih iznačajki metodom polinomijalne regresije nije zamjetno poboljšalo model, čini se da čak radi lošije za polinomijalnu regresiju 3. stupnja.

Možemo izvesti istrenirane modele i skaliranja za upotrebu predviđanja niza podataka.

In [17]:
from pickle import dump

# Spremanje modela i skalera za daljnje korištenje
dump(reg_1, open('reg_1.pkl', 'wb'))
dump(reg_2, open('reg_2.pkl', 'wb'))
dump(reg_3, open('reg_3.pkl', 'wb'))

dump(scaler_1, open('scaler_1.pkl', 'wb'))
dump(scaler_2, open('scaler_2.pkl', 'wb'))
dump(scaler_3, open('scaler_3.pkl', 'wb'))